# 문제 1. t-검정과 z-검정의 비교

In [1]:
from scipy.stats import t, norm
import numpy as np

mu = 0
sigma = 1
n = norm(loc = mu, scale = sigma)
alpha = 0.05

def compareTandZ(num):
    count = 0
    for i in range(500):
        sample = n.rvs(size=num)
        zlen = 2 * n.ppf(1-alpha/2) * (sigma/np.sqrt(num))
        tlen = 2 * t.ppf(1-alpha/2, num-1) * np.std(sample, ddof=1) / np.sqrt(num)
        if(tlen > zlen): count += 1
    return count/500
    
print("1-1)", compareTandZ(30))

print("1-2)")
print("n=", 10, "비율:", compareTandZ(10))
print("n=", 30, "비율:", compareTandZ(30))
print("n=", 50, "비율:", compareTandZ(50))
print("n=", 100, "비율:", compareTandZ(100))
print("n=", 500, "비율:", compareTandZ(500))
print("n=", 1000, "비율:", compareTandZ(1000))

1-1) 0.58
1-2)
n= 10 비율: 0.654
n= 30 비율: 0.57
n= 50 비율: 0.568
n= 100 비율: 0.524
n= 500 비율: 0.518
n= 1000 비율: 0.532


## 결과해석

표본의 크기를 parameter로 받아 t-검정시의 신뢰구간의 너비가 z-검정 시의 신뢰구간의 너비가 큰 경우의 비율을 return하는 함수를 정의하였다.  

n 값이 커지면서 비율이 0.5에 근접하는 것은 표본의 크기가 커짐에 따라  
t분포가 정규분포에 근사되며, 표본표준편차가 모표준편차에 근사되는 것을 알 수 있다.

# 문제 2. 등분산성 검정의 이해

In [2]:
from scipy.stats import ttest_rel
import numpy as np

score_bef = np.array([18, 21, 16, 22, 19, 24, 17, 21, 23, 18, 14, 16, 16, 19, 18, 20, 12, 22, 15, 17])
score_aft = np.array([22, 25, 17, 24, 16, 29, 20, 23, 19, 20, 15, 15, 18, 26, 18, 24, 18, 25, 19, 16])

print("2-1)", ttest_rel(score_bef, score_aft))
print("2-2)", ttest_rel(score_aft, score_bef))

print("2-3)", end = ' ')
print("2-1)에서의 F 통계량:", np.var(score_bef, ddof=1)/np.var(score_aft, ddof=1), end = '\t')
print("2-2)에서의 F 통계량:", np.var(score_aft, ddof=1)/np.var(score_bef, ddof=1))

2-1) Ttest_relResult(statistic=-3.231252665580312, pvalue=0.004394965993185664)
2-2) Ttest_relResult(statistic=3.231252665580312, pvalue=0.004394965993185664)
2-3) 2-1)에서의 F 통계량: 0.603291260584758	2-2)에서의 F 통계량: 1.6575741525423726


## 결과해석

2-1)과 2-2)의 결과를 비교해봤을 때, 검정통계량은 절댓값은 같고 부호만 다르며 유의확률은 같다.  
이는 대응비교는 각 쌍에서 얻은 관측값의 차로 두 모평균의 차에 대한 추론을 다루는 문제이므로,
두 집단의 순서를 반대로 할 경우, 분석 대상이 되는 각 쌍에서의 관측값 차의 부호가 반대가 되어,
평균은 부호만 다르고 분포 형태는 동일하여 표준편차는 같기 때문이다.

2-3) 두 경우의 F 통계량은 역수 관계이다.  
이는 F분포 특성상 확률변수 F가 자유도 (v1, v2)인 F-분포를 따를 경우, 1/F는 자유도 (v2, v1)인 F-분포를 따르기 때문이다.

# 문제 3. run10samp.txt

In [3]:
import pandas as pd
from scipy.stats import ttest_ind, f

run10samp = pd.read_csv('run10samp.txt', sep=" ")

def var_test(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    S1 = np.var(sample1, ddof=1)
    S2 = np.var(sample2, ddof=1)
    dfn = n1-1
    dfd = n2-1
    F = S1 / S2
    pval = 2*min(f.cdf(F, dfn = dfn, dfd = dfd), 1-f.cdf(F, dfn=dfn, dfd=dfd))
    print("F test to compare two variances: F = %s, p-value = %s" % (round(F,5), round(pval,5)))
    if(pval > 0.05): return True
    else: return False
    
group1 = run10samp[run10samp['age'] >= 35]['time']
group2 = run10samp[run10samp['age'] < 35]['time']

print("3-1)", ttest_ind(group1, group2, equal_var = var_test(group1, group2)))

run10samp_M = run10samp[run10samp['gender'] == "M"]

group3 = run10samp_M[run10samp_M['age'] >= 35]['time']
group4 = run10samp_M[run10samp_M['age'] < 35]['time']

print("3-2)", ttest_ind(group3, group4, equal_var = False))

group_DC = run10samp[run10samp['state'] == "DC"]['time']
group_MD = run10samp[run10samp['state'] == "MD"]['time']
group_NY = run10samp[run10samp['state'] == "NY"]['time']

print("3-3)", "DC:", group_DC.size, "MD:", group_MD.size, "NY:", group_NY.size)

print("3-4)", ttest_ind(group_NY, group_MD, equal_var=False))

print("3-5)", ttest_ind(group_DC, group_MD, equal_var=False))



F test to compare two variances: F = 0.55545, p-value = 0.05161
3-1) Ttest_indResult(statistic=-2.007702832701262, pvalue=0.04742789007338827)
3-2) Ttest_indResult(statistic=-0.03298318822670364, pvalue=0.9738412378625644)
3-3) DC: 26 MD: 18 NY: 6
3-4) Ttest_indResult(statistic=1.0087981102438832, pvalue=0.3371545300547275)
3-5) Ttest_indResult(statistic=1.1008589237183508, pvalue=0.27862209919272724)


## 결과해석

3-1)에서 두 그룹의 등분산 여부 검정 결과, 검정 통계량은 0.55545이고 유의확률은 0.05161이었다. 따라서 주어진 유의수준 5%에서 두 모집단의 분산이 같다는 귀무가설을 기각할 수 없다.  
등분산일 때의 이표본에 의한 모평균 비교를 할 때, 검정통계량은 -2.0077이고 유의확률은 0.047427이었다. 따라서 주어진 유의수준 5%에서 35살 이상과 35살 미만의 선수들의 평균 완주 기록에 차이가 있다는 귀무가설을 기각할 수 있다.  

3-2) 등분산이 아닐 때의 이표본에 의한 모평균 비교를 할 때, 검정통계량은 -0.03298이고 유의확률은 0.973841이었다. 따라서 주어진 유의수준 5%에서 35살 이상 남성 선수과 35살 미만 남성 선수의 평균 완주 기록에 차이가 있다는 귀무가설을 기각할 수 없다.  

3-3) 출신 지역이 DC, MD, NY인 선수의 수는 각각 26명, 18명, 6명이다.  

3-4) 등분산이 아닐 때의 이표본에 의한 모평균 비교를 할 때, 검정통계량은 1.008798이고 유의확률은 0.337154이었다. 따라서 주어진 유의수준 5%에서 출신 지역이 NY인 선수와 출신 지역이 MD인 선수의 평균 완주 기록에 차이가 있다는 귀무가설을 기각할 수 없다.  

3-5) 등분산이 아닐 때의 이표본에 의한 모평균 비교를 할 때, 검정통계량은 1.1008589이고 유의확률은 0.278622이었다. 따라서 주어진 유의수준 5%에서 출신 지역이 DC인 선수와 출신 지역이 MD인 선수의 평균 완주 기록에 차이가 있다는 귀무가설을 기각할 수 없다.  

이 때, 3-4)와 3-5)의 유의확률은 각각 0.337154와 0.278622이다.
3-5)의 유의확률이 더 작은 이유는 T 분포의 특성 상 자유도가 높을 수록 동일한 유의수준에 대해서 값이 작아지기 때문이다.  
이는 비교 대상이 되는 집단의 표본 수가 클 수록 자유도가 높아지기 때문에,  
NY와 MD를 비교할 때의 유의확률이 DC와 MD를 비교할 때의 유의확률보다 크게 관찰된다.
 

